<a href="https://colab.research.google.com/github/gianguyen7/Recommender-System-ALS/blob/main/Implementation_Platform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Dictionary
'''
reviewerID - ID of the reviewer, e.g. A2SUAM1J3GNN3B
asin - ID of the product, e.g. 0000013714
reviewerName - name of the reviewer
helpful - helpfulness rating of the review, e.g. 2/3
reviewText - text of the review
overall - rating of the product
summary - summary of the review
unixReviewTime - time of the review (unix time)
reviewTime - time of the review (raw)
'''

'\nreviewerID - ID of the reviewer, e.g. A2SUAM1J3GNN3B\nasin - ID of the product, e.g. 0000013714\nreviewerName - name of the reviewer\nhelpful - helpfulness rating of the review, e.g. 2/3\nreviewText - text of the review\noverall - rating of the product\nsummary - summary of the review\nunixReviewTime - time of the review (unix time)\nreviewTime - time of the review (raw)\n'

In [2]:
#Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

# %matplotlib inline
plt.style.use("ggplot")

import sklearn
from sklearn.decomposition import TruncatedSVD

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [162]:
#Read in the Data
data = pd.read_csv("/content/gdrive/My Drive/DS 340W/musical_instruments.csv")
data.head(5)

,reviewerID,asin,reviewerName,helpful/0,helpful/1,reviewText,overall,summary,unixReviewTime,reviewTime
0,A2IBPI20UZIR0U,1384719342,"cassandra tu ""Yeah, well, that's just like, u...",0,0,"Not much to write about here, but it does exac...",5,good,1393545600,"02 28, 2014"
1,A14VAT5EAX3D9S,1384719342,Jake,13,14,The product does exactly as it should and is q...,5,Jake,1363392000,"03 16, 2013"
2,A195EZSQDW3E21,1384719342,"Rick Bennette ""Rick Bennette""",1,1,The primary job of this device is to block the...,5,It Does The Job Well,1377648000,"08 28, 2013"
3,A2C00NNG1ZQQG2,1384719342,"RustyBill ""Sunday Rocker""",0,0,Nice windscreen protects my MXL mic and preven...,5,GOOD WINDSCREEN FOR THE MONEY,1392336000,"02 14, 2014"
4,A94QU4C90B1AX,1384719342,SEAN MASLANKA,0,0,This pop filter is great. It looks and perform...,5,No more pops when I record my vocals.,1392940800,"02 21, 2014"


In [140]:
#Checking for NA values
data.isnull().sum()

reviewerID         0
asin               0
reviewerName      27
helpful/0          0
helpful/1          0
reviewText         7
overall            0
summary            0
unixReviewTime     0
reviewTime         0
dtype: int64

In [141]:
#Dropping all rows with NA
data = data.dropna()

In [142]:
data.shape

(10227, 10)

In [143]:
#Defining popular instruments in the dataset
popular_products = pd.DataFrame(data.groupby('asin')['overall'].count())
most_popular = popular_products.sort_values('overall', ascending=False)
most_popular.head(10)

,overall
asin,
B003VWJ2K8,163
B0002E1G5C,143
B0002F7K7Y,116
B003VWKPHC,114
B0002H0A3S,93
B0002CZVXM,74
B0006NDF8A,71
B0009G1E0K,69
B0002E2KPC,68


In [144]:
#Plotting most popular instruments
most_popular.head(30).plot(kind = "bar")

<IPython.core.display.Javascript object>

EDA

In [145]:
ratingCount =  data.groupby(['overall'])
ratingCount = ratingCount['overall'].count()
ratingCount = ratingCount.nlargest(10, 'first')
ratingCount = px.bar(ratingCount, x = 'overall', title = 'Top Rating Counts', width = 800, height = 800)
ratingCount.update_layout(yaxis_title = "overall", xaxis_title = "count")
ratingCount.show()

In [163]:
#Filtering out products and users > 5
min_ratings = 5
filter_instruments = data['asin'].value_counts() > min_ratings
filter_instruments = filter_instruments[filter_instruments].index.tolist()

min_user_ratings = 5
filter_users = data['reviewerID'].value_counts() > min_user_ratings
filter_users = filter_users[filter_users].index.tolist()

df_new = data[(data['asin'].isin(filter_instruments)) & (data['reviewerID'].isin(filter_users))]
print('The original data frame shape:\t{}'.format(data.shape))
print('The new data frame shape:\t{}'.format(df_new.shape))

The original data frame shape:	(10261, 10)
The new data frame shape:	(6754, 10)


**Implementation**

In [17]:
!pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for vecstack: filename=vecstack-0.4.0-py3-none-any.whl size=19877 sha256=b7fcc519e5fc69d347f73b8263fc7219ff8967cd8ae17330aba3e1b33bb1ba92
  Stored in directory: /root/.cache/pip/wheels/28/fe/0c/fe8e43660e3316d7ce204e59a79a72246c0ae9b6c5c79841c8
Successfully built vecstack


In [164]:
from surprise import Reader, Dataset, SVD, NMF, accuracy
from surprise import NormalPredictor
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNWithZScore
from surprise import KNNBaseline
from surprise import BaselineOnly
from surprise import SVDpp
from surprise import SlopeOne
from surprise import CoClustering
from surprise.model_selection.validation import cross_validate
from surprise.model_selection import KFold, train_test_split, GridSearchCV
from collections import defaultdict


reader = Reader()
data = Dataset.load_from_df(df_new[['reviewerID','asin', 'overall']], reader)
svd = SVD()

In [165]:
def precision_recall_at_k(predictions, k=20, threshold=3):
    """Return precision and recall at k metrics for each user"""

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(
            ((true_r >= threshold) and (est >= threshold))
            for (est, true_r) in user_ratings[:k]
        )

        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. We here set it to 0.

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. We here set it to 0.

        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    return precisions, recalls

In [166]:
benchmark = []
# Iterate over all algorithms
for algorithm in [SVD(), SVDpp(), SlopeOne(), NMF(), NormalPredictor(), KNNBaseline(), KNNBasic(), KNNWithMeans(), KNNWithZScore(), BaselineOnly(), CoClustering()]:
    # Perform cross validation
    results = cross_validate(algorithm, data, measures=['RMSE'], cv=3, verbose=False)
    
    # Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)
    
pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse') 

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...


,test_rmse,fit_time,test_time
Algorithm,,,
BaselineOnly,0.855121,0.012152,0.010793
SVDpp,0.859825,0.822785,0.073813
SVD,0.865258,0.308572,0.019789
KNNBaseline,0.944773,0.039069,0.054674
CoClustering,0.973602,0.247144,0.012157
KNNWithZScore,0.974539,0.067581,0.059261
KNNBasic,0.981259,0.027075,0.053165
KNNWithMeans,0.985327,0.047052,0.052352
SlopeOne,1.047539,0.027975,0.023334


In [167]:
#BaselineOnly gives us the best rmse so we will train on that
print('Using ALS')
bsl_options = {'method': 'als',
               'n_epochs': 5,
               'reg_u': 12,
               'reg_i': 5
               }
algo = BaselineOnly(bsl_options=bsl_options)
cross_validate(algo, data, measures=['RMSE'], cv=3, verbose=False)

Using ALS
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...


{'test_rmse': array([0.85357152, 0.88931386, 0.83441571]),
 'fit_time': (0.007031917572021484,
  0.018422365188598633,
  0.015942096710205078),
 'test_time': (0.01830291748046875,
  0.011049032211303711,
  0.011741161346435547)}

In [185]:
trainset, testset = train_test_split(data, test_size=0.25)
algo = BaselineOnly(bsl_options=bsl_options)
predictions = algo.fit(trainset).test(testset)
a_baseline = accuracy.rmse(predictions)
print(a_baseline)

precisions, recalls = precision_recall_at_k(predictions, k=20, threshold=3)
print("Precision for BaselineOnly: ")
print(sum(prec for prec in precisions.values()) / len(precisions))
print("Recall for BaselineOnly: ")
print(sum(rec for rec in recalls.values()) / len(recalls))

Estimating biases using als...
RMSE: 0.8309
0.8308930000212323
Precision for BaselineOnly: 
0.9568439010824455
Recall for BaselineOnly: 
0.9797843665768194


In [186]:
def get_Iu(uid):
    """ return the number of items rated by given user
    args: 
      uid: the id of the user
    returns: 
      the number of items rated by the user
    """
    try:
        return len(trainset.ur[trainset.to_inner_uid(uid)])
    except ValueError: # user was not part of the trainset
        return 0
    
def get_Ui(iid):
    """ return number of users that have rated given item
    args:
      iid: the raw id of the item
    returns:
      the number of users that have rated the item.
    """
    try: 
        return len(trainset.ir[trainset.to_inner_iid(iid)])
    except ValueError:
        return 0
    
df = pd.DataFrame(predictions, columns=['uid', 'iid', 'rui', 'est', 'details'])
df['Iu'] = df.uid.apply(get_Iu)
df['Ui'] = df.iid.apply(get_Ui)
df['err'] = abs(df.est - df.rui)
best_predictions = df.sort_values(by='err')[:10]
worst_predictions = df.sort_values(by='err')[-10:]

In [187]:
best_predictions.head(10)

,uid,iid,rui,est,details,Iu,Ui,err
1166,A1Q94IXHEGQDV7,B001W99HE8,5.0,5.000000,{'was_impossible': False},5,16,0.000000
1012,A2RI4QE6EO29CA,B001W99HE8,5.0,5.000000,{'was_impossible': False},4,16,0.000000
512,A26M5O53PHZTKN,B004N0MKN8,4.0,3.997094,{'was_impossible': False},6,15,0.002906
95,AHCJAJO18KRP1,B001G3TJXO,5.0,4.987815,{'was_impossible': False},6,7,0.012185
774,A3EXWV8FNSSFL6,B007T8OGLK,4.0,3.977949,{'was_impossible': False},11,12,0.022051
1348,A319B090A2POEB,B000KIPUQG,4.0,4.027201,{'was_impossible': False},10,2,0.027201
1091,A1MTN9XWJSA5PI,B0002D01KO,4.0,3.964966,{'was_impossible': False},4,4,0.035034
367,A2W8OGJDV7TCMT,B0025V1REU,4.0,3.961605,{'was_impossible': False},4,3,0.038395
104,A23NSKTMSPPBTR,B0002II6V0,4.0,3.961267,{'was_impossible': False},4,6,0.038733
1575,AHF7SNSZPRNCE,B001REJX1U,4.0,4.041654,{'was_impossible': False},5,2,0.041654


In [188]:
worst_predictions.head(10)

,uid,iid,rui,est,details,Iu,Ui,err
575,A1IB42LRNA9QLW,B000EEEYCW,1.0,4.563039,{'was_impossible': False},6,7,3.563039
1405,A3MTR7P9PYAALX,B0002GYW4C,1.0,4.574695,{'was_impossible': False},7,18,3.574695
1373,ANZGC4YCEQC6W,B000A2HDXA,1.0,4.605207,{'was_impossible': False},5,5,3.605207
1527,A2W7M00WSI4JND,B0002M6CVC,1.0,4.645626,{'was_impossible': False},3,29,3.645626
837,A306YUQBEB2R9T,B000EELBQ4,1.0,4.698352,{'was_impossible': False},12,6,3.698352
825,A3G6P4SJ8SYH9F,B003VWJ2K8,1.0,4.712099,{'was_impossible': False},9,82,3.712099
417,A1LS2UXHG1BYRG,B0009G1E0K,1.0,4.744034,{'was_impossible': False},6,29,3.744034
40,A1MI9FDCNB3CMR,B0002GXF8Q,1.0,4.748965,{'was_impossible': False},6,17,3.748965
1462,A2053ZJUGCKUA5,B0087UPSLQ,1.0,4.769301,{'was_impossible': False},9,4,3.769301
1116,A2YSBRBYSI0524,B005BSOVRY,1.0,4.774298,{'was_impossible': False},7,3,3.774298


In [ ]:
#need to do preprocessing, data cleaning, add 1-2 more methods aside from svd to do ensemble, and hyperparameter tuning